In [11]:
from time import time
import polars as pl
import pandas as pd
import numpy as np
import pyarrow

import matplotlib.pyplot as plt
import seaborn as sns

df = pl.read_csv(r"./datasets/2019-Nov-1M.csv")

In [12]:
%%time

# With groupby we can group elements by columns' values and perform some aggregation operations

df.groupby("brand").agg(
    pl.count().alias("num_items")
)

CPU times: total: 0 ns
Wall time: 15 ms


brand,num_items
str,u32
"""baldessarini""",18
"""adile""",174
"""canon""",1653
"""prestigio""",1191
"""burberry""",20
"""biema""",3
"""babyono""",3
"""unikum""",2
"""neptun""",3


In [13]:
df.select([pl.count()])

count
u32
1000000


In [14]:
%%time

# The same result could be sorted as follows

df.groupby("brand").agg([
    pl.count().alias("num_items_per_brand")
]).sort("num_items_per_brand", descending=True)

CPU times: total: 15.6 ms
Wall time: 23.8 ms


brand,num_items_per_brand
str,u32
null,147043
"""samsung""",126248
"""apple""",99213
"""xiaomi""",72601
"""huawei""",23976
"""lucente""",15625
"""oppo""",13833
"""lg""",13254
"""bosch""",12282


In [15]:
%%time

# Perform count and percentage of instances

df.groupby("brand").agg([
    pl.count()
]).sort("count", descending=True).with_columns([
    (pl.col("count") / pl.col("count").sum()).alias("percentage")
])


CPU times: total: 0 ns
Wall time: 14.6 ms


brand,count,percentage
str,u32,f64
null,147043,0.147043
"""samsung""",126248,0.126248
"""apple""",99213,0.099213
"""xiaomi""",72601,0.072601
"""huawei""",23976,0.023976
"""lucente""",15625,0.015625
"""oppo""",13833,0.013833
"""lg""",13254,0.013254
"""bosch""",12282,0.012282


In [16]:
%%time
df['event_type'].value_counts()

CPU times: total: 0 ns
Wall time: 13 ms


event_type,counts
str,u32
"""view""",967225
"""cart""",14958
"""purchase""",17817


In [17]:
%%time
df.groupby('event_type').agg(pl.count())

CPU times: total: 15.6 ms
Wall time: 66.1 ms


event_type,count
str,u32
"""purchase""",17817
"""cart""",14958
"""view""",967225


In [23]:
%%time

#Groupby on two columns and sorting
df_brand_event = df.groupby(["brand", "event_type"]).agg(pl.count()).sort(["count"], descending=True)

df_brand_event.head()

CPU times: total: 15.6 ms
Wall time: 19 ms


brand,event_type,count
str,str,u32
null,"""view""",145478
"""samsung""","""view""",116794
"""apple""","""view""",91848
"""xiaomi""","""view""",69501
"""huawei""","""view""",22773


In [22]:
%%time

#The pivot method make possible to fix a column as index and to expand the values of a specific column as columns using the values of the target column to fill and distribute across the new pivot columns created

df_brand_event.pivot(values="count",
                     index=["brand"],
                     columns=["event_type"])

C:\Users\Giacomo\AppData\Local\Temp\ipykernel_19620\3921812555.py:1: DeprecationWarning: In a future version of polars, the default `aggregate_function` will change from `'first'` to `None`. Please pass `'first'` to keep the current behaviour, or `None` to accept the new one.
  df_brand_event.pivot(values="count",


brand,view,cart,purchase
str,u32,u32,u32
null,145478,185,1380
"""samsung""",116794,5202,4252
"""apple""",91848,3916,3449
"""xiaomi""",69501,1820,1280
"""huawei""",22773,715,488
"""lucente""",15363,3,259
"""oppo""",13164,373,296
"""lg""",12900,149,205
"""bosch""",12109,61,112


In [25]:
%%time

#It is possible to combine sorting with pivot operations

df_brand_event.pivot(values="count",
                     index=["brand"],
                     columns=["event_type"]).sort(["purchase", "cart"], descending=True)

CPU times: total: 0 ns
Wall time: 5.01 ms


<timed eval>:3: DeprecationWarning: In a future version of polars, the default `aggregate_function` will change from `'first'` to `None`. Please pass `'first'` to keep the current behaviour, or `None` to accept the new one.


brand,view,cart,purchase
str,u32,u32,u32
"""samsung""",116794,5202,4252
"""apple""",91848,3916,3449
null,145478,185,1380
"""xiaomi""",69501,1820,1280
"""huawei""",22773,715,488
"""cordiant""",10908,271,369
"""oppo""",13164,373,296
"""lucente""",15363,3,259
"""lg""",12900,149,205


In [28]:
%%time

# We can perform operation on every pivot columns and keep the original ones too

agg_performance = df_brand_event.pivot(values="count",
                     index=["brand"],
                     columns=["event_type"]).sort(
    ["purchase", "cart"], descending=True).with_columns([
    (pl.col("cart")/pl.col('view')).alias('cart_by_views'),
    (pl.col("purchase")/pl.col('cart')).alias('buy_by_cart'),
    (pl.col("purchase")/pl.col('view')).alias('buy_by_views')
])

agg_performance.head()


CPU times: total: 0 ns
Wall time: 10 ms


<timed exec>:3: DeprecationWarning: In a future version of polars, the default `aggregate_function` will change from `'first'` to `None`. Please pass `'first'` to keep the current behaviour, or `None` to accept the new one.


brand,view,cart,purchase,cart_by_views,buy_by_cart,buy_by_views
str,u32,u32,u32,f64,f64,f64
"""samsung""",116794,5202,4252,0.04454,0.817378,0.036406
"""apple""",91848,3916,3449,0.042636,0.880746,0.037551
null,145478,185,1380,0.001272,7.459459,0.009486
"""xiaomi""",69501,1820,1280,0.026187,0.703297,0.018417
"""huawei""",22773,715,488,0.031397,0.682517,0.021429


In [29]:
# Filter and Select operations on pivot aggregation

agg_performance.filter(pl.col('buy_by_cart')<1).select(
    ['cart_by_views','buy_by_cart','buy_by_views']
).to_pandas().quantile([0,0.25,0.5,0.75,0.99,1.0])

,cart_by_views,buy_by_cart,buy_by_views
0.00,0.004556,0.210526,0.002278
0.25,0.013736,0.500000,0.008621
0.50,0.022059,0.666667,0.015184
0.75,0.032888,0.809524,0.020260
0.99,0.091796,0.923736,0.043895
1.00,0.106481,0.939560,0.046296


In [30]:
%%time

# Groupby on multiple columns

df_user_brand_event = df.groupby(["user_id", "brand", "event_type"]).agg([pl.count()])
df_user_brand_event

CPU times: total: 109 ms
Wall time: 62.6 ms


user_id,brand,event_type,count
i64,str,str,u32
561587266,"""creed""","""view""",1
518085591,"""lg""","""view""",1
513200477,"""oasis""","""view""",6
551977964,"""sony""","""view""",4
561587266,"""chanel""","""view""",1
512376444,"""aoteli""","""view""",2
548645138,"""stels""","""view""",1
566280567,"""huawei""","""view""",1
532572658,"""xiaomi""","""view""",4
